In [101]:
import requests
import re
import os
from os import path

In [62]:
url = "https://api.github.com/repos/Netflix/Hystrix/commits/00485dd245317622d8eb24fd5e48327aefae9f9b"
csha = '00485dd245317622d8eb24fd5e48327aefae9f9b'

In [6]:
r = requests.get(url)
print(r.content)

b'{"sha":"00485dd245317622d8eb24fd5e48327aefae9f9b","node_id":"MDY6Q29tbWl0Njc2NjU1ODowMDQ4NWRkMjQ1MzE3NjIyZDhlYjI0ZmQ1ZTQ4MzI3YWVmYWU5Zjli","commit":{"author":{"name":"Matt Jacobs","email":"mjacobs@netflix.com","date":"2015-02-23T20:12:24Z"},"committer":{"name":"Matt Jacobs","email":"mjacobs@netflix.com","date":"2015-02-23T20:12:24Z"},"message":"Make HystrixCommandProperties backwards-compatible for classes which extend it","tree":{"sha":"f222acafe563b842edcac894aecd26d9674763b4","url":"https://api.github.com/repos/Netflix/Hystrix/git/trees/f222acafe563b842edcac894aecd26d9674763b4"},"url":"https://api.github.com/repos/Netflix/Hystrix/git/commits/00485dd245317622d8eb24fd5e48327aefae9f9b","comment_count":0,"verification":{"verified":false,"reason":"unsigned","signature":null,"payload":null}},"url":"https://api.github.com/repos/Netflix/Hystrix/commits/00485dd245317622d8eb24fd5e48327aefae9f9b","html_url":"https://github.com/Netflix/Hystrix/commit/00485dd245317622d8eb24fd5e48327aefae9f9b",

In [8]:
result = r.json()

dict

In [70]:
# get time
commit_time = result['commit']['committer']['date']
print(commit_time)

2015-02-23T20:12:24Z


In [60]:
def get_java_url(file_paths):
    
    java_check = r"\.java$"
    
    file_urls = []
    
    for index, item in enumerate(file_paths):
        file_url = item['raw_url']    
        print(index, file_url)
        print(re.match(r"^.*\.java$", str(file_url)))
        if re.match(r"^.*\.java$", str(file_url)):
            file_urls.append(file_url)
    
    return file_urls

In [61]:
# get file path
file_paths = result['files']
print(type(file_paths))

file_urls = get_java_url(file_paths)

print(file_urls)

<class 'list'>
0 https://github.com/Netflix/Hystrix/raw/00485dd245317622d8eb24fd5e48327aefae9f9b/hystrix-core/src/main/java/com/netflix/hystrix/HystrixCommandProperties.java
<_sre.SRE_Match object; span=(0, 156), match='https://github.com/Netflix/Hystrix/raw/00485dd245>
['https://github.com/Netflix/Hystrix/raw/00485dd245317622d8eb24fd5e48327aefae9f9b/hystrix-core/src/main/java/com/netflix/hystrix/HystrixCommandProperties.java']


In [69]:
def get_path(file_url):
    path_re = ''
    result  = re.search(r'^.*' + csha + '(.*java$)', file_url)
    print(result.group(1))
    return result.group(1)

print(get_path(file_urls[0]))

/hystrix-core/src/main/java/com/netflix/hystrix/HystrixCommandProperties.java
/hystrix-core/src/main/java/com/netflix/hystrix/HystrixCommandProperties.java


In [85]:
def get_previous_commit(time, file_url):
    
    path = get_path(file_url)
    
    url = 'https://api.github.com/repos/Netflix/Hystrix/commits?path='+path+'&until='+time
    
    req = requests.get(url)
    
    result = req.json()
    commit_id = result[1]['sha']
    print(commit_id)
    
    return commit_id

commit_id = get_previous_commit(commit_time, file_urls[0])

/hystrix-core/src/main/java/com/netflix/hystrix/HystrixCommandProperties.java
727cbebe11f5ab86bdc6312e070f03122f27a638


In [109]:
print(commit_id)

727cbebe11f5ab86bdc6312e070f03122f27a638


In [115]:
def get_prev_url(commit_id, file_url):
    previous_url = "https://api.github.com/repos/Netflix/Hystrix/commits/" + commit_id
    print(previous_url)

    req = requests.get(previous_url)
    prev_filename = get_path(file_url)
    prev_filename = prev_filename.split('/', 1)[1]

    for item in req.json()['files']:
        # print(item['filename'])
        if item['filename'] == prev_filename:
            prev_file_url = item['raw_url']
            # print(prev_file_url)
    
    return prev_file_url

prev_url = get_prev_url(commit_id, file_urls[0])
print(prev_url)

https://api.github.com/repos/Netflix/Hystrix/commits/727cbebe11f5ab86bdc6312e070f03122f27a638
/hystrix-core/src/main/java/com/netflix/hystrix/HystrixCommandProperties.java
https://github.com/Netflix/Hystrix/raw/727cbebe11f5ab86bdc6312e070f03122f27a638/hystrix-core/src/main/java/com/netflix/hystrix/HystrixCommandProperties.java


In [96]:
filename = get_path(file_urls[0]).split('/')[-1]
print(filename)

/hystrix-core/src/main/java/com/netflix/hystrix/HystrixCommandProperties.java
HystrixCommandProperties.java


In [103]:
pwd = os.getcwd()
print(pwd)

prev_dir = pwd + '/' + filename.split('.')[0] + 'prev.java'


/Users/apple/min/Research Project/Software commit/Automatic Classification
/Users/apple/min/Research Project/Software commit/Automatic Classification/HystrixCommandPropertiesprev.java


In [116]:
print(prev_url)

https://github.com/Netflix/Hystrix/raw/727cbebe11f5ab86bdc6312e070f03122f27a638/hystrix-core/src/main/java/com/netflix/hystrix/HystrixCommandProperties.java


In [117]:
pwd = os.getcwd()

prev_dir = pwd + '/' + filename.split('.')[0] + 'prev.java'
curr_dir = pwd + '/' + filename.split('.')[0] + 'curr.java'

if not path.exists(prev_dir):
    with open(filename.split('.')[0] + 'prev.java','wb') as f:
        req = requests.get(prev_url)
        f.write(req.content)

if not path.exists(curr_dir):
    with open(filename.split('.')[0] + 'curr.java','wb') as f:
        req = requests.get(file_urls[0])
        f.write(req.content)

In [126]:
import subprocess

pre_filename = filename.split('.')[0] + 'prev.java'
curr_filename = filename.split('.')[0] + 'curr.java'

cmd = ['gumtree', 'cluster', pre_filename, curr_filename]

output = subprocess.Popen(cmd, stdout=subprocess.PIPE ).communicate()[0]

print(output)

with open('output.txt','wb') as f:
    f.write(output)

b'New cluster:\nINS MethodInvocation to ReturnStatement at 0\n------------\nINS MethodInvocation to ReturnStatement at 0\nINS SimpleName: executionIsolationThreadTimeoutInMilliseconds to MethodInvocation at 0\n\nNew cluster:\nDEL SimpleName: executionTimeoutInMilliseconds\n------------\nDEL SimpleName: executionTimeoutInMilliseconds\n\n'
